In [4]:
#dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import time

In [5]:
# # Mars Headlines
#url and get page with requests
url = 'https://mars.nasa.gov/news/'
response = requests.get(url)

In [6]:
#get your soup (object)
headlines_soup = bs(response.text, 'lxml')

In [7]:
#scrape the latest news headline text and get rid of the white space
headline = headlines_soup.find('div', class_ = 'content_title').text.strip()

In [8]:
#print it to make sure it works
print(headline)

NASA's Curiosity Mars Rover Finds a Clay Cache


In [9]:
#scrape the blurb text that goes with the headline - a.k.a. the first paragraph without white space 
news = headlines_soup.find('div', class_ = 'rollover_description_inner').text.strip()

In [10]:
#make sure that one worked too...
print(news)

The rover recently drilled two samples, and both showed the highest levels of clay ever found during the mission.


In [11]:
# # Get your featured image of Mars
# Import Selenium & Splinter and set the chromedriver path (I'm on windows)
from selenium import webdriver
from splinter import Browser
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [12]:
#visit the URL
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

In [13]:
#click the button to get to the full image
browser.click_link_by_id('full_image')

In [12]:
#hold 2 seconds to let it load
time.sleep(2)

In [14]:
#scrape browser into soup and find full resolution image of Mars
#btw, there is no "largesize" image link
#Bring up the image
html = browser.html
image_soup = bs(html, "lxml")
img_url = image_soup.find('img', class_ = 'fancybox-image')['src']

In [15]:
#append the url snippet to make it the full url of the image 
featured_img_url = "https://www.jpl.nasa.gov" + img_url

In [16]:
#verify we have the url
featured_img_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17761_ip.jpg'

In [17]:
#on to getting the latest tweet about the weather on Mars
url = 'https://twitter.com/marswxreport?lang=en'
response = requests.get(url)

In [18]:
# create soup object
weather_soup = bs(response.text, 'html.parser')

In [19]:
#grab all the tweets in case the first one isn't the one you want
mars_weather_tweet = weather_soup.select(".tweet-text")

In [19]:
# find the first actual weather tweet (they start with "sol")
for tweet in mars_weather_tweet:
    if tweet.text.strip().startswith('Sol'):
        mars_weather = tweet.text.strip()

In [20]:
print(mars_weather_tweet)

[<p class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text" data-aria-label-part="0" lang="en">InSight sol 195 (2019-06-14) low -105.6ºC (-158.1ºF) high -23.8ºC (-10.8ºF)
winds from the SSE at 4.3 m/s (9.7 mph) gusting to 14.5 m/s (32.5 mph)
pressure at 7.60 hPa<a class="twitter-timeline-link u-hidden" data-pre-embedded="true" dir="ltr" href="https://t.co/3LNNz5uXCd">pic.twitter.com/3LNNz5uXCd</a></p>, <p class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text" data-aria-label-part="0" lang="en">InSight sol 194 (2019-06-13) low -104.4ºC (-155.9ºF) high -23.5ºC (-10.3ºF)
winds from the SSE at 4.5 m/s (10.0 mph) gusting to 17.0 m/s (38.1 mph)
pressure at 7.60 hPa</p>, <p class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text" data-aria-label-part="0" lang="en">InSight sol 193 (2019-06-12) low -103.7ºC (-154.7ºF) high -23.0ºC (-9.4ºF)
winds from the SSE at 4.4 m/s (9.8 mph) gusting to 15.7 m/s (35.1 mph)
pressure at 7.60 hPa<a class="twitter-timeline-

In [21]:
[x.text for x in mars_weather_tweet]

['InSight sol 195 (2019-06-14) low -105.6ºC (-158.1ºF) high -23.8ºC (-10.8ºF)\nwinds from the SSE at 4.3 m/s (9.7 mph) gusting to 14.5 m/s (32.5 mph)\npressure at 7.60 hPapic.twitter.com/3LNNz5uXCd',
 'InSight sol 194 (2019-06-13) low -104.4ºC (-155.9ºF) high -23.5ºC (-10.3ºF)\nwinds from the SSE at 4.5 m/s (10.0 mph) gusting to 17.0 m/s (38.1 mph)\npressure at 7.60 hPa',
 'InSight sol 193 (2019-06-12) low -103.7ºC (-154.7ºF) high -23.0ºC (-9.4ºF)\nwinds from the SSE at 4.4 m/s (9.8 mph) gusting to 15.7 m/s (35.1 mph)\npressure at 7.60 hPapic.twitter.com/EhemsIhPLv',
 'InSight sol 192 (2019-06-11) low -104.9ºC (-156.8ºF) high -23.6ºC (-10.6ºF)\nwinds from the W at 4.3 m/s (9.7 mph) gusting to 15.4 m/s (34.4 mph)\npressure at 7.60 hPapic.twitter.com/qacCeZLc6e',
 'InSight sol 191 (2019-06-10) low -104.9ºC (-156.9ºF) high -21.6ºC (-6.9ºF)\nwinds from the SSE at 4.4 m/s (9.9 mph) gusting to 15.2 m/s (33.9 mph)\npressure at 7.60 hPa',
 'Mars has a new 16 mile wide crater. Hopefully @NASAIn

In [22]:
# # Mars Facts
#visit the space facts site
url = 'https://space-facts.com/mars/'
browser.visit(url)

In [23]:
# find the html table and convert to pandas dataframe
mars_df = pd.read_html(url)
mars_df = (mars_df[0])

In [24]:
#set column headers 
mars_df.columns = ["Description", "Value"]
# mars_df = mars_df.set_index("Description")
# I decided not to reset the index because, to me, it looks messier to have the column headers in two mis-aligned cells

In [25]:
mars_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [26]:
mars_df = mars_df.to_html(classes='mars')
table_data = mars_df.replace('\n', ' ')
table_data

'<table border="1" class="dataframe mars">   <thead>     <tr style="text-align: right;">       <th></th>       <th>Description</th>       <th>Value</th>     </tr>   </thead>   <tbody>     <tr>       <th>0</th>       <td>Equatorial Diameter:</td>       <td>6,792 km</td>     </tr>     <tr>       <th>1</th>       <td>Polar Diameter:</td>       <td>6,752 km</td>     </tr>     <tr>       <th>2</th>       <td>Mass:</td>       <td>6.42 x 10^23 kg (10.7% Earth)</td>     </tr>     <tr>       <th>3</th>       <td>Moons:</td>       <td>2 (Phobos &amp; Deimos)</td>     </tr>     <tr>       <th>4</th>       <td>Orbit Distance:</td>       <td>227,943,824 km (1.52 AU)</td>     </tr>     <tr>       <th>5</th>       <td>Orbit Period:</td>       <td>687 days (1.9 years)</td>     </tr>     <tr>       <th>6</th>       <td>Surface Temperature:</td>       <td>-153 to 20 °C</td>     </tr>     <tr>       <th>7</th>       <td>First Record:</td>       <td>2nd millennium BC</td>     </tr>     <tr>       <th>8</t